# 구글 웨이티 정보 수집

In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from tqdm import tqdm
from webdriver_manager.chrome import ChromeDriverManager
import time
from datetime import datetime

찾을 매장 불러오기

In [2]:
# path = "C:/Users/yis82/OneDrive/DOCUME~1-LAPTOP-0LF8B10V-3815/Desktop/Lunch Lab/crawling_data/data"
df = pd.read_excel("data/2-1_naver_매장_기본정보_크롤링.xlsx")


In [3]:
df = df[:3]
rest_name = df['검색어']

In [10]:
class GoogleMapsScraper:
    def __init__(self, dataframe):
        self.df = dataframe
        # Initialize the WebDriver
        self.driver = webdriver.Chrome()
        self.base_url = "https://www.google.com/maps"
    
    def input_function(self, n):
        search_name = self.df['검색어'].iloc[n] #검색어 가져오기
        print(search_name)
        self.driver.get("https://www.google.com/maps")
        wait = WebDriverWait(self.driver, 10)  # Wait up to 10 seconds
        search_box = wait.until(EC.element_to_be_clickable((By.ID, "searchboxinput")))
        search_box.clear()  # Clear the search box before inputting text
        search_box.send_keys(search_name)
        search_box.send_keys(Keys.ENTER)
        time.sleep(5)  # Wait for search results to load
        #검색하고 첫번째 매장 클릭
        try:
            button = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'hfpxzc')))
            button.click()
            print("첫번째 매장 클릭 완료")  # 첫번째 매장 클릭 성공 메시지
        except Exception as e:
            print("매장이 바로 검색되었습니다.: ")  # 클릭 실패 메시지

    def get_store_hours(self, n):
        percentages = []

        # 진행사항을 알기 위해서 tqdm 사용
        for i in tqdm(range(len(self.df)), desc='진행 중', position=0, leave=True):
            search_name = self.df['검색어'][i]
            self.input_function(i)  # Call input_function to perform search
            html = self.driver.page_source
            soup = BeautifulSoup(html, 'html.parser')

            # tTyJAb 클래스 안에 있는 요소들을 찾기
            elements = soup.find_all(class_='tTyJAb')

            # 각 요소에서 aria-label > 그래프 내의 숫자만 가져오기 
            for element in elements:
                aria_label = element.parent.get('aria-label')
                if aria_label:
                    percentage = aria_label.split('%')[0].split()[-1]
                    percentages.append(percentage)

        return percentages


    def make_dataframe(self, store_names, percentages):
        # 오늘 날짜 가져와서 오늘부터 시작하여 월요일부터 일요일까지의 요일 리스트 만들기
        today = datetime.now()
        weekday_number = today.weekday()
        weekdays = ["월요일", "화요일", "수요일", "목요일", "금요일", "토요일", "일요일"]
        days_of_week = weekdays[weekday_number:] + weekdays[:weekday_number]
        hours = [f"{i + 6}시" for i in range(18)]
        data = {}

        index = 0
        for store_name in store_names:
            store_data = {day: {} for day in days_of_week}
            for day in days_of_week:
                hour_data = {}
                for hour in hours:
                    if index < len(percentages):
                        hour_data[hour] = percentages[index]
                        index += 1
                    else:
                        hour_data[hour] = None  # Fill with None if no data available
                store_data[day] = hour_data
            data[store_name] = store_data

        return data



In [23]:
# Adjust the path to your WebDriver here
driver_path = path

store_names = df['검색어']
# Initialize the scraper with the DataFrame and the driver path
scraper = GoogleMapsScraper(df,)

# Execute the scraping process
percentages  = scraper.get_store_hours(0)

days_df = scraper.make_dataframe(store_names, percentages)


진행 중:   0%|          | 0/3 [00:00<?, ?it/s]

역삼로 215, 신촌설렁탕
첫번째 매장 클릭 완료


진행 중:  33%|███▎      | 1/3 [00:06<00:13,  6.95s/it]

압구정로32길 32 캘리스코 사보텐
첫번째 매장 클릭 완료


진행 중:  67%|██████▋   | 2/3 [00:13<00:06,  6.88s/it]

논현로 841 피오렌티나
매장이 바로 검색되었습니다.: 


진행 중: 100%|██████████| 3/3 [00:30<00:00, 10.17s/it]


In [26]:
days_df

{'역삼로 215, 신촌설렁탕': {'토요일': {'6시': '0',
   '7시': '0',
   '8시': '0',
   '9시': '0',
   '10시': '0',
   '11시': '38',
   '12시': '67',
   '13시': '74',
   '14시': '57',
   '15시': '45',
   '16시': '37',
   '17시': '38',
   '18시': '35',
   '19시': '32',
   '20시': '30',
   '21시': '27',
   '22시': '0',
   '23시': '0'},
  '일요일': {'6시': '0',
   '7시': '0',
   '8시': '0',
   '9시': '0',
   '10시': '0',
   '11시': '59',
   '12시': '100',
   '13시': '96',
   '14시': '72',
   '15시': '49',
   '16시': '38',
   '17시': '37',
   '18시': '61',
   '19시': '89',
   '20시': '96',
   '21시': '89',
   '22시': '0',
   '23시': '0'},
  '월요일': {'6시': '0',
   '7시': '0',
   '8시': '0',
   '9시': '0',
   '10시': '0',
   '11시': '57',
   '12시': '77',
   '13시': '59',
   '14시': '32',
   '15시': '18',
   '16시': '18',
   '17시': '37',
   '18시': '62',
   '19시': '76',
   '20시': '79',
   '21시': '59',
   '22시': '0',
   '23시': '0'},
  '화요일': {'6시': '0',
   '7시': '0',
   '8시': '0',
   '9시': '0',
   '10시': '0',
   '11시': '54',
   '12시': '81',
   '13시': '71',


## 지선

In [4]:
df = pd.read_excel("data/2-1_naver_매장_기본정보_크롤링.xlsx")
df = df[:3]
rest_name = df['검색어']

In [22]:

class GoogleMapsScraper:
    def __init__(self, dataframe):
        self.df = dataframe
        # WebDriver 초기화
        self.driver = webdriver.Chrome(ChromeDriverManager().install())
        self.base_url = "https://www.google.com/maps"
    
    def input_function(self, n):
        # 검색어 가져오기
        search_name = self.df['검색어'].iloc[n]
        print(search_name)
        self.driver.get("https://www.google.com/maps")
        wait = WebDriverWait(self.driver, 10)  # 최대 10초까지 대기
        search_box = wait.until(EC.element_to_be_clickable((By.ID, "searchboxinput")))
        search_box.clear()  # 텍스트를 입력하기 전에 검색 상자를 지웁니다.
        search_box.send_keys(search_name)
        search_box.send_keys(Keys.ENTER)
        time.sleep(5)  # 검색 결과가 로드될 때까지 대기
        # 첫 번째 매장을 클릭합니다.
        try:
            button = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'hfpxzc')))
            button.click()
            print("첫 번째 매장 클릭 완료")  # 첫 번째 매장 클릭 성공 메시지
        except Exception as e:
            print("매장이 바로 검색되었습니다.: ")  # 클릭 실패 메시지

    def get_store_hours(self, n):
        percentages = []
        # 진행률을 표시하기 위해 tqdm을 사용합니다.
        for i in tqdm(range(len(self.df)), desc='진행 중', position=0, leave=True):
            search_name = self.df['검색어'][i]
            self.input_function(i)  # 검색을 수행하려면 input_function을 호출합니다.
            html = self.driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            # 'tTyJAb' 클래스를 포함하는 요소 찾기
            elements = soup.find_all(class_='tTyJAb')
            # 각 요소에서 aria-label을 찾아 그래프 내의 숫자만 가져옵니다.
            for element in elements:
                aria_label = element.parent.get('aria-label')
                if aria_label:
                    percentage = aria_label.split('%')[0].split()[-1]
                    percentages.append(percentage)
        return percentages

    def make_dataframe(self, store_names, percentages):
        # 오늘 날짜 가져와서 # 오늘부터 시작하여 월요일부터 일요일까지의 요일 리스트 만들기
        today = datetime.now()
        weekday_number = today.weekday()
        weekdays = ["월요일", "화요일", "수요일", "목요일", "금요일", "토요일", "일요일"]
        days_of_week = weekdays[weekday_number:] + weekdays[:weekday_number]
        hours = [f"{i + 6}시" for i in range(18)]
        data = {}
        index = 0
        for store_name in store_names:
            store_data = {day: {} for day in days_of_week}
            for day in days_of_week:
                hour_data = {}
                for hour in hours:
                    if index < len(percentages):
                        hour_data[hour] = percentages[index]
                        index += 1
                    else:
                        hour_data[hour] = None  # 데이터가 없는 경우 None으로 채웁니다.
                store_data[day] = hour_data
            data[store_name] = store_data

        return data


In [23]:
# # Adjust the path to your WebDriver here
# driver_path = path

store_names = df['검색어']
# Initialize the scraper with the DataFrame and the driver path
scraper = GoogleMapsScraper(df,)

# Execute the scraping process
percentages  = scraper.get_store_hours(0)

days_df = scraper.make_dataframe(store_names, percentages)

days_df

진행 중:   0%|          | 0/3 [00:00<?, ?it/s]

역삼로 215, 신촌설렁탕


진행 중:  33%|███▎      | 1/3 [00:08<00:16,  8.31s/it]

첫 번째 매장 클릭 완료
압구정로32길 32 캘리스코 사보텐


진행 중:  67%|██████▋   | 2/3 [00:15<00:07,  7.64s/it]

첫 번째 매장 클릭 완료
논현로 841 피오렌티나
매장이 바로 검색되었습니다.: 


진행 중: 100%|██████████| 3/3 [00:33<00:00, 11.07s/it]


{'역삼로 215, 신촌설렁탕': {'월요일': {'6시': '0',
   '7시': '0',
   '8시': '0',
   '9시': '0',
   '10시': '0',
   '11시': '38',
   '12시': '67',
   '13시': '74',
   '14시': '57',
   '15시': '45',
   '16시': '37',
   '17시': '38',
   '18시': '35',
   '19시': '32',
   '20시': '30',
   '21시': '27',
   '22시': '0',
   '23시': '0'},
  '화요일': {'6시': '0',
   '7시': '0',
   '8시': '0',
   '9시': '0',
   '10시': '0',
   '11시': '59',
   '12시': '100',
   '13시': '96',
   '14시': '72',
   '15시': '49',
   '16시': '38',
   '17시': '37',
   '18시': '61',
   '19시': '89',
   '20시': '96',
   '21시': '89',
   '22시': '0',
   '23시': '0'},
  '수요일': {'6시': '0',
   '7시': '0',
   '8시': '0',
   '9시': '0',
   '10시': '0',
   '11시': '57',
   '12시': '77',
   '13시': '59',
   '14시': '32',
   '15시': '18',
   '16시': '18',
   '17시': '37',
   '18시': '62',
   '19시': '76',
   '20시': '79',
   '21시': '59',
   '22시': '0',
   '23시': '0'},
  '목요일': {'6시': '0',
   '7시': '0',
   '8시': '0',
   '9시': '0',
   '10시': '0',
   '11시': '54',
   '12시': '81',
   '13시': '71',


#### -------------------------------------------------------------------------------------------

# Test

# 근데 보니깐 맨 첫번째 식당 쪽에 광고하는 식당이 붙어버리면 다른 식당 페이지로 넘어감


In [2]:
df = pd.read_excel("data/2-1_naver_매장_기본정보_크롤링.xlsx")
df = df[:5]
rest_name = df['검색어']

In [3]:
from selenium import webdriver
driver = webdriver.Chrome('')
base_url = "https://www.google.com/maps"


In [4]:
percentages = []
store_names =[] 
for n in range(len(df)) :
    search_name = df['검색어'].iloc[n]
    store_names.append(search_name)
    print(search_name)
    driver.get("https://www.google.com/maps")
    # wait = WebDriverWait(driver, 10)  # 최대 10초까지 대기
    # search_box = wait.until(EC.element_to_be_clickable((By.ID, "searchboxinput")))
    search_box = driver.find_element(By.ID, "searchboxinput")
    search_box.clear()  # 텍스트를 입력하기 전에 검색 상자를 지웁니다.
    search_box.send_keys(search_name)
    search_box.send_keys(Keys.ENTER)
    time.sleep(5)  # 검색 결과가 로드될 때까지 대기
    # 첫 번째 매장을 클릭합니다.
    try:
        button_ele = driver.find_elements(By.CLASS_NAME,'hfpxzc')
        button = button_ele[0]
        button.click()
        print("첫 번째 매장 클릭 완료")  # 첫 번째 매장 클릭 성공 메시지
    except Exception as e:
        print("매장이 바로 검색되었습니다.: ")  # 클릭 실패 메시지
        
    time.sleep(3)
    html = driver.page_source
    soup = BeautifulSoup(html,'html.parser')
    # 'tTyJAb' 클래스를 포함하는 요소 찾기
    # elements = soup.find_all(class_='g2BVhd')
    elements = soup.find_all(class_='tTyJAb')
    # 각 요소에서 aria-label을 찾아 그래프 내의 숫자만 가져옵니다.
    for element in elements:
        aria_label = element.parent.get('aria-label')
        if aria_label:
            percentage = aria_label.split('정도 ')[1].split('%')[0]
            # percentage = aria_label
            percentages.append(percentage)
 

역삼로 215, 신촌설렁탕
첫 번째 매장 클릭 완료
압구정로32길 32 캘리스코 사보텐
첫 번째 매장 클릭 완료
논현로 841 피오렌티나
매장이 바로 검색되었습니다.: 
도산대로46길 21 압구정변강쇠떡볶이
첫 번째 매장 클릭 완료
논현로 429 마포만두
매장이 바로 검색되었습니다.: 


In [43]:
def make_dataframe(store_names, percentages):
    # 요일 리스트
    weekdays = ["월요일", "화요일", "수요일", "목요일", "금요일", "토요일", "일요일"]
    
    # 오늘의 요일 구하기
    today_index = datetime.datetime.today().weekday()  # 0 = 월요일, ..., 6 = 일요일
    
    # 오늘의 요일이 먼저 오도록 요일 리스트 재정렬
    reordered_weekdays = weekdays[today_index:] + weekdays[:today_index]

    hours = [f"{i}시" for i in range(6, 24)]  # 6시부터 23시까지

    data = {}
    index = 0
    for store_name in store_names:
        store_data = {day: {} for day in reordered_weekdays}
        for day in reordered_weekdays:
            for hour in hours:
                if index < len(percentages):
                    store_data[day][hour] = percentages[index]
                    index += 1
                else:
                    store_data[day][hour] = None  # 데이터가 없는 경우 None으로 채웁니다.
        data[store_name] = store_data

    return data

# 사용 예시
store_names = ["Store A", "Store B"]
percentages = [i for i in range(126 * len(store_names))]  # 임의의 데이터 생성
data = make_dataframe(store_names, percentages)
print(data)


{'Store A': {'목요일': {'6시': 0, '7시': 1, '8시': 2, '9시': 3, '10시': 4, '11시': 5, '12시': 6, '13시': 7, '14시': 8, '15시': 9, '16시': 10, '17시': 11, '18시': 12, '19시': 13, '20시': 14, '21시': 15, '22시': 16, '23시': 17}, '금요일': {'6시': 18, '7시': 19, '8시': 20, '9시': 21, '10시': 22, '11시': 23, '12시': 24, '13시': 25, '14시': 26, '15시': 27, '16시': 28, '17시': 29, '18시': 30, '19시': 31, '20시': 32, '21시': 33, '22시': 34, '23시': 35}, '토요일': {'6시': 36, '7시': 37, '8시': 38, '9시': 39, '10시': 40, '11시': 41, '12시': 42, '13시': 43, '14시': 44, '15시': 45, '16시': 46, '17시': 47, '18시': 48, '19시': 49, '20시': 50, '21시': 51, '22시': 52, '23시': 53}, '일요일': {'6시': 54, '7시': 55, '8시': 56, '9시': 57, '10시': 58, '11시': 59, '12시': 60, '13시': 61, '14시': 62, '15시': 63, '16시': 64, '17시': 65, '18시': 66, '19시': 67, '20시': 68, '21시': 69, '22시': 70, '23시': 71}, '월요일': {'6시': 72, '7시': 73, '8시': 74, '9시': 75, '10시': 76, '11시': 77, '12시': 78, '13시': 79, '14시': 80, '15시': 81, '16시': 82, '17시': 83, '18시': 84, '19시': 85, '20시': 86, '21시': 87, '22시'

In [44]:
days_df = make_dataframe(store_names,percentages)

In [45]:
data = {
    '업체명': [],
    '요일': [],
    '시간': [],
    '밀집도': []
}

for place, info in days_df.items():
    for day, hours in info.items():
        for hour, density in hours.items():
            data['업체명'].append(place)
            data['요일'].append(day)
            data['시간'].append(hour)
            data['밀집도'].append(density)

result_df = pd.DataFrame(data)


In [46]:
result_df

,업체명,요일,시간,밀집도
0,Store A,목요일,6시,0
1,Store A,목요일,7시,1
2,Store A,목요일,8시,2
3,Store A,목요일,9시,3
4,Store A,목요일,10시,4
...,...,...,...,...
247,Store B,수요일,19시,247
248,Store B,수요일,20시,248
249,Store B,수요일,21시,249
250,Store B,수요일,22시,250


In [ ]:
pd.set_option('display.max_rows', None)
result_df